In [0]:
#########################################Notebook Arguments############################################# 
dbutils.widgets.text("job_config_identifier" ,"" ,"job_config_identifier")
jobConfigIdentifier = dbutils.widgets.get("job_config_identifier")
print(f"jobConfigIdentifier - {jobConfigIdentifier}")

dbutils.widgets.text("batch_set_category" ,"" ,"batch_set_category")
batchSetCategory = dbutils.widgets.get("batch_set_category")
print(f"batchSetCategory - {batchSetCategory}")

dbutils.widgets.text("batch_set_src_sys_name" ,"" ,"batch_set_src_sys_name")
batchSetSrcSysName = dbutils.widgets.get("batch_set_src_sys_name")
print(f"batchSetSrcSysName - {batchSetSrcSysName}")

dbutils.widgets.text("batch_set_src_sys_cd" ,"" ,"batch_set_src_sys_cd")
batchSetSrcSysCd = dbutils.widgets.get("batch_set_src_sys_cd")
print(f"batchSetSrcSysCd - {batchSetSrcSysCd}")

dbutils.widgets.text("job_identifier" ,"" ,"job_identifier")
jobIdentifier = dbutils.widgets.get("job_identifier")
print(f"jobIdentifier - {jobIdentifier}")

dbutils.widgets.text("logging_mode" ,"" ,"logging_mode")
loggingMode = dbutils.widgets.get("logging_mode").upper()
print(f"loggingMode - {loggingMode}")

In [0]:
%run /DATA_ENGINEERING/2_AXIOM/1_SHARED/UTILITIES/pre_job_execution_utility

In [0]:
import datetime
import os
from pyspark.sql.types import *
from pyspark.sql.functions import *
notebookJobId = 0 if str(dbutils.notebook.entry_point.getDbutils().notebook().getContext().jobId()) == 'None' else dbutils.notebook.entry_point.getDbutils().notebook().getContext().jobId().get()
#Stage completion flag for identifying success of previous stage in the notebook 
previousStageCompletionFlag = True
# Default Error msg , will be overwritten with error msg in exception block , used as notebook exit message
errorMsg =''
#Get notebook Name
notebookName=os.path.basename(dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get())

In [0]:
configDetails=configUtilityObject.getConfigFromTable(f"{jobConfigIdentifier}")
configDetailsDefault={
  "param_src_sys_cd":"WIN",
  "param_src_sys_id":"27",  
  "param_cfm_schema":"dw_xle_fdp",
  "param_win_schema":"staging_wins_fdp",
  "param_interface_schema":"dw_xle_sz_interface_fdp",
  "param_sz_delta_schema":"dw_xle_sz_delta_fdp",
  "param_sz_schema":"dw_xle_sz_fdp",
  "param_src_balance_sql":"Select count(1) as record_count from dw_xle_fdp.t_d_xll_policy where l_plcy_src_sys_cd = 'WIN' and dw_row_active_ind='Y'",
  "param_tgt_balance_sql":"Select count(1) as record_count from dw_xle_sz_delta_fdp.t_d_xll_policy where l_plcy_src_sys_cd = 'WIN' and dw_row_active_ind='Y'"}

#Merge Default values with values from table.
configDetails={**configDetailsDefault, **configDetails}
loggerObject.info(f"**************Config From Table************************")
loggerObject.info(f"{configDetails}")

#Initialize variables
SRC_SYS_CD=configDetails["param_src_sys_cd"]
SRC_SYS_ID=configDetails["param_src_sys_id"]
CFM_SCHEMA=configDetails["param_cfm_schema"]
SZ_DELTA_SCHEMA=configDetails["param_sz_delta_schema"]
SZ_SCHEMA=configDetails["param_sz_schema"]
WIN_SCHEMA=configDetails["param_win_schema"]
INTERFACE_SCHEMA=configDetails["param_interface_schema"]
srcBalanceSql=configDetails["param_src_balance_sql"]
tgtBalanceSql=configDetails["param_tgt_balance_sql"]

In [0]:
#DB\Table Info
INTERFACE_TABLE="t_d_xll_policy_win_int"
TGT_SZ_TBL="t_d_xll_policy"

#Batch Info
batchName=jobIdentifier
batchSrcDbName=CFM_SCHEMA
batchTgtDbName=SZ_DELTA_SCHEMA
batchTgtTblName=TGT_SZ_TBL

#Paths
INTERFACE_SCHEMA_PATH=f"{getAdlsContainer('STANDARDIZE',configDetails)}/{INTERFACE_SCHEMA}"

In [0]:
stageName = 'Opening Batch Stage'
if previousStageCompletionFlag == True:
  try:
    loggerObject.info(f"########## {stageName} ##########")
    batchBalanceObject.openBatch(batchSetCategory, batchSetSrcSysName, batchName, batchSrcDbName, batchTgtDbName, batchTgtTblName)
    batchKey = batchBalanceObject.getBatchInProgress(batchSetCategory, batchSetSrcSysName, batchName).collect()[0]['batch_key']
  except Exception as e:
    previousStageCompletionFlag = False
    errorMsg = f"FAILURE | Notebook run for {notebookName} - jobId: {notebookJobId} ,failed at : {stageName} "
    loggerObject.info(f"!!!!!!! Exception While {stageName} !!!!!!!")
    loggerObject.exception(e)
    print(e)
else:
    loggerObject.info(f"################# Skipping {stageName} #################")

In [0]:
stageName = 'Read From Source'
if previousStageCompletionFlag == True:
  try:    
    loggerObject.info(f"########## {stageName} ##########")  
    srcFilterCondition=f"l_plcy_src_sys_cd='{SRC_SYS_CD}' and ltrim(rtrim(dw_row_active_ind)) = 'Y'"
    df_lkp_xle_plcy = spark.sql(f""" SELECT DISTINCT
                                                    pol.plcy_textnk
                                                    from {CFM_SCHEMA}.t_b_xle_ri_contract_layer_policy bridge
                                                    join {CFM_SCHEMA}.t_d_xle_policy pol
                                                    on  pol.plcy_id = bridge.plcy_id
                                                    and pol.dw_row_active_ind = 'Y'
                                                    join {CFM_SCHEMA}.t_d_xle_ri_contract_layer ricl
                                                    on ricl.ri_cont_layer_id = bridge.ri_cont_layer_id
                                                    and ricl.dw_row_active_ind = 'Y'
                                                    where 
                                                        bridge.dw_row_active_ind = 'Y' AND
                                                        pol.plcy_src_sys_cd = '{SRC_SYS_CD}' AND
                                                        ri_cont_captive_ind = 'Yes' """)
                                              
    df_lkp_t_wins_dwxp010 = spark.sql(f""" select distinct concat('{SRC_SYS_CD}~', policy ,'~', CAST(effdte AS varchar(8))) AS plcy_txtnk,flagi6 AS l_plcy_wins_flagi6,l_plcy_customer_base_unit,l_plcy_gross_revenue_cd,l_plcy_form,l_plcy_print_flag,win_user_id_add,win_src_sys_cd,l_last_endr_expr_dt,l_plcy_last_endr_descr from (select policy,effdte,flagi6,ftrfl6 as l_plcy_customer_base_unit,grsrev as l_plcy_gross_revenue_cd, polfom as l_plcy_form,polprt as l_plcy_print_flag,idadd as win_user_id_add, src_sys_id as win_src_sys_cd,covend as l_last_endr_expr_dt,edmnts as l_plcy_last_endr_descr,ROW_NUMBER() OVER (PARTITION BY policy, effdte ORDER BY edsno DESC) AS win_row_nbr from  {WIN_SCHEMA}.t_wins_dwxp010) a where win_row_nbr=1 """)
    
    dfSrc=spark.sql(f"Select * from {CFM_SCHEMA}.{TGT_SZ_TBL} where {srcFilterCondition}")\
                .join(df_lkp_xle_plcy,col("l_plcy_textnk") == df_lkp_xle_plcy.plcy_textnk,"left")\
                .join(df_lkp_t_wins_dwxp010,col("l_plcy_textnk") ==  df_lkp_t_wins_dwxp010.plcy_txtnk,"left")\
                .drop("plcy_txtnk")\
                .withColumn("l_principal_plcy_ind",expr("CASE WHEN plcy_textnk IS NULL THEN 'Non-Principal' ELSE 'Principal' END"))\
                .withColumn("dw_batch_id",lit(batchKey))\
                .withColumn("l_real_or_mirror_plcy_ind",lit('Real'))\
                .withColumn("l_territory_scope_descr",lit(' '))
                
    

    dfSrc.createOrReplaceTempView("finalTransformation")
    
  except Exception as e:
    previousStageCompletionFlag = False
    errorMsg = f"FAILURE | Notebook run for {notebookName} - jobId: {notebookJobId} ,failed at : {stageName}"
    batchBalanceObject.closeBatch( batchSetCategory, batchSetSrcSysName, batchName, 'F')
    loggerObject.info(f"!!!!!!! Exception While {stageName} !!!!!!!")
    loggerObject.exception(e)
    print(e)
else:
    loggerObject.info(f"################# Skipping {stageName} #################")
    

In [0]:
stageName = 'Interface Write'
if previousStageCompletionFlag == True:
  try:
    loggerObject.info(f"########## {stageName} ##########")
    sqlInterface = f""" SELECT
                              l_plcy_key,
                              l_plcy_id,
                              l_plcy_textnk,
                              l_plcy_hashnk,
                              l_plcy_hashfull,
                              l_plcy_nbr,
                              l_plcy_eff_dt,
                              l_plcy_expr_dt,
                              l_indemnity_pd,
                              l_branch_cd,
                              l_branch_nme,
                              l_broker_cd,
                              l_broker_nme,
                              TRIM(l_captive_ind )AS l_captive_ind,
                              l_company_cd,
                              l_company_nme,
                              l_cont_type_cd,
                              l_cont_type_descr,
                              l_ctry_of_settlement_cd,
                              l_dept_cd,
                              l_dept_nme,
                              l_event_dt,
                              l_plcy_ny_free_trd_zn_class_cd,
                              l_acct_analysis_cd,
                              l_inc_prod_dir_ri_cd,
                              l_inc_prod_dir_ri_descr,
                              l_insured_cd,
                              l_insured_nme,
                              l_insurer_cd,
                              l_insurer_nme,
                              l_inwards_reinsurance_flag,
                              l_lead_yn,
                              l_lta_eff_dt,
                              l_lta_expr_dt,
                              TRIM(l_lta_ind) AS l_lta_ind,
                              l_mainline_cd,
                              l_mainline_descr,
                              l_mgmt_triangle_cd,
                              l_mrk_reg_cd,
                              l_new_rnwl_ind,
                              l_num_days_to_cancel,
                              l_plcy_binder_mga_ind,
                              l_plcy_cancel_dt,
                              l_plcy_src_sys_cd,
                              l_plcy_status_cd,
                              l_plcy_status_descr,
                              l_plcy_true_expr_dt,
                              l_plcy_uw_year,
                              l_prod_office_cd,
                              l_prod_office_descr,
                              l_product_cd,
                              l_product_descr,
                              l_program_nbr,
                              l_quote_nbr,
                              l_ri_particpant_grp_flg,
                              l_sic_cd,
                              l_sic_descr,
                              l_risk_class_cd,
                              l_risk_class_descr,
                              l_sub_risk_class_cd,
                              l_sub_risk_class_descr,
                              l_syndicate_cd,
                              l_syndicate_ssr_cd,
                              l_territory_scope,
                              l_trd_of_bus_cd,
                              l_trd_of_bus_descr,
                              l_underwriter_cd,
                              l_underwriter_nme,
                              genius_fkgenintpolnum,
                              genius_fkgenintpolseq,
                              TRIM(genius_nad_r_advcf3) AS genius_nad_r_advcf3,
                              genius_newoldeconomyflag,
                              dw_row_active_ind,
                              dw_row_start_dt,
                              dw_row_end_dt,
                              dw_batch_id,
                              dw_create_dt,
                              dw_last_update_dt,
                              l_brokerage_comm_pct,
                              l_plcy_title,
                              l_risk_band_cd,
                              l_risk_band_descr,
                              l_prim_layer,
                              l_ctry_cd,
                              l_unique_mrkt_ref_identifier,
                              frame_cost_center_cd,
                              frame_lnk_back_to_first_section_seq,
                              frame_lnk_back_to_prior_section_seq,
                              frame_plcy_risk_nbr,
                              frame_plcy_declaration_nbr,
                              frame_plcy_risk_section_sid,
                              frame_underwriting_ref_identifier,
                              genius_fklocalreportcode,
                              l_plcy_term_in_days,
                              l_plcy_cancel_reason_cd,
                              l_plcy_cancel_reason_descr,
                              l_plcy_ny_free_trd_zn_stat_cd,
                              l_predominant_epmi_pol_sec_cd,
                              genius_nad_r_adnucd,
                              TRIM(l_plcy_risk_purch_grp_ind) AS l_plcy_risk_purch_grp_ind,
                              l_plcy_ny_free_trd_zn_class_descr,
                              l_plcy_ny_free_trd_zn_stat_descr,
                              TRIM(l_plcy_multi_year_ind) AS l_plcy_multi_year_ind,
                              frame_prior_underwriting_ref_identifier,
                              l_prior_policy_renewed_from_identifier,
                              l_legacy_prior_policy_renewed_from_identifier,
                              l_plcy_ceded_basis_cvrg_cd,
                              l_plcy_ceded_basis_cvrg_descr,
                              l_plcy_acctg_behaviour_cd,
                              l_plcy_reins_flag_cd,
                              l_plcy_reins_flag_descr,
                              l_plcy_agent_nme,
                              l_plcy_agent_email,
                              TRIM(l_plcy_authorisation_status_ind) AS l_plcy_authorization_status_ind,
                              l_plcy_deregulation_type_cd,
                              l_plcy_deregulation_type_descr,
                              l_us_listed AS l_plcy_class_us_listed_ind,
                              l_ctry_of_settlement_descr,
                              l_plcy_primary_cvrg_scheme_cd,
                              l_plcy_primary_cvrg_scheme_descr,
                              l_plcy_xl_risk_ind,
                              l_plcy_axa_ind,
                              l_plcy_trust_fund_cd,
                              l_plcy_trust_fund_descr,
                              l_plcy_foreign_insurance_legislation_4_cd,
                              l_plcy_foreign_insurance_legislation_4_descr,
                              l_plcy_deleted_ind,
                              TRIM(l_plcy_risk_section_deleted_ind) AS l_plcy_risk_section_deleted_ind,
                              TRIM(l_plcy_div_cd) AS l_plcy_div_cd,
                              l_plcy_ml_cd,
                              win_program_cd,
                              l_plcy_revenue_synergy_cd,
                              l_plcy_revenue_synergy_descr,
                              l_plcy_ilu_cd,
                              l_plcy_eea_ind_short_cd,
                              l_plcy_eea_ind_descr,
                              genius_auditlog_event_cd,
                              genius_auditlog_event_descr,
                              l_plcy_innovation_class_ind,
                              TRIM(l_plcy_minimization_ind) AS l_plcy_minimization_ind,
                              l_principal_plcy_ind,
                              l_real_or_mirror_plcy_ind,
                              l_territory_scope_descr,
                              l_plcy_wins_flagi6,
                              l_plcy_customer_base_unit,
                              l_plcy_gross_revenue_cd,
                              l_plcy_form,
                              '' as l_mirror_orig_plcy_nbr,
                              '' as l_plcy_mirror_type_descr,
                              '' as l_plcy_cedant_ref,
                              coalesce(l_plcy_src_type,'') as l_plcy_src_type,
                              coalesce(l_plcy_print_flag,'') as l_plcy_print_flag,
                              coalesce(win_user_id_add,'') as win_user_id_add,
                              coalesce(win_src_sys_cd,'') as win_src_sys_cd,
                              coalesce(l_plcy_last_endr_descr,'') as l_plcy_last_endr_descr,
                              TO_DATE(CAST(l_last_endr_expr_dt as STRING),'yyyyMMdd') as l_last_endr_expr_dt
                              from finalTransformation"""
    dfInterface=spark.sql(sqlInterface)
    dfInterface.write.format("parquet").mode("overWrite").option("path", f"{INTERFACE_SCHEMA_PATH}/{INTERFACE_TABLE}/").saveAsTable(f"{INTERFACE_SCHEMA}.{INTERFACE_TABLE}")
  except Exception as e:
    previousStageCompletionFlag = False
    errorMsg = f"FAILURE | Notebook run for {notebookName} - jobId: {notebookJobId} ,failed at : {stageName}"
    batchBalanceObject.closeBatch( batchSetCategory, batchSetSrcSysName, batchName, 'F')
    loggerObject.info(f"!!!!!!! Exception While {stageName} !!!!!!!")
    loggerObject.exception(e)
    print(e)
else:
    loggerObject.info(f"################# Skipping {stageName} #################")

In [0]:
tgtTableSchema=SZ_DELTA_SCHEMA
tgtTableName=TGT_SZ_TBL
tgtFilterCondition=f"tgt.l_plcy_src_sys_cd='{SRC_SYS_CD}' and tgt.dw_row_active_ind='Y'"
srcFilterCondition=f"src.l_plcy_src_sys_cd='{SRC_SYS_CD}' and src.dw_row_active_ind='Y'"
joinColumnList=['l_plcy_id']
seqPrefixValue=f"99{scdObject.getUniqSeqValueFromAbcDb(abcDbObj, batchKey, tgtTableSchema, TGT_SZ_TBL, SRC_SYS_CD)}"
newInsertCondition="src.l_plcy_id IS NOT NULL and tgt.l_plcy_id IS NULL"
newInsertExplicitColumnMap={
  "l_plcy_key_orig":"src.l_plcy_key",
  "dw_last_update_dt_orig":"src.dw_last_update_dt",
  "dw_row_active_ind":"'Y'",
  "dw_row_end_dt":"to_timestamp('9999-12-31 00:00:00.000')",
  "dw_create_dt":"current_timestamp()",
  "dw_last_update_dt":"current_timestamp()"}

confChangeCondition="src.l_plcy_id=tgt.l_plcy_id and src.l_plcy_key != tgt.l_plcy_key_orig"									
confChangeInsertExplicitColumnMap={
  "l_plcy_key_orig":"src.l_plcy_key",
  "dw_last_update_dt_orig":"src.dw_last_update_dt",
  "dw_row_active_ind": "'Y'",
  "dw_row_start_dt":"current_timestamp()",
  "dw_row_end_dt":"to_timestamp('9999-12-31 00:00:00.000')",
  "dw_create_dt":"current_timestamp()",
  "dw_last_update_dt":"current_timestamp()"}
confChangeUpdateExplicitColumnMap={"dw_row_end_dt":"current_timestamp()","dw_row_active_ind":"'N'","dw_last_update_dt":"current_timestamp()"}

axiomChangeDetectMd5ColumnList1=['l_principal_plcy_ind','l_real_or_mirror_plcy_ind','l_territory_scope_descr','l_plcy_wins_flagi6','l_plcy_customer_base_unit','l_plcy_gross_revenue_cd','l_plcy_form','l_mirror_orig_plcy_nbr','l_plcy_mirror_type_descr','l_plcy_cedant_ref','l_plcy_print_flag','win_user_id_add','win_src_sys_cd','l_last_endr_expr_dt','l_plcy_last_endr_descr']
axiomChangeDetectMd5ColumnList2=[]
axiomChangeCondition="src.l_plcy_id=tgt.l_plcy_id AND src.l_plcy_key = tgt.l_plcy_key_orig and src.axiomChangeDetectMd5Column1 != tgt.axiomChangeDetectMd5Column1"
axiomChangeInsertExplicitColumnMap={
  "l_plcy_key":"new_surrogate_key",
  "l_plcy_key_orig":"src.l_plcy_key",
  "dw_last_update_dt_orig":"src.dw_last_update_dt",
  "dw_create_dt":"current_timestamp()",
  "dw_last_update_dt":"current_timestamp()",
  "dw_row_active_ind": "'Y'",
  "dw_row_start_dt":"current_timestamp()",
  "dw_row_end_dt":"to_timestamp('9999-12-31 00:00:00.000')"
}
axiomChangeUpdateExplicitColumnMap={"dw_row_end_dt":"current_timestamp()","dw_row_active_ind":"'N'","dw_last_update_dt":"current_timestamp()"}

confHardStopCondition="src.l_plcy_id=tgt.l_plcy_id AND src.l_plcy_key = tgt.l_plcy_key_orig and src.axiomChangeDetectMd5Column1 = tgt.axiomChangeDetectMd5Column1 and src.dw_last_update_dt > tgt.dw_last_update_dt_orig"
confHardStopExplicitColumnMap={
  "l_plcy_key":"tgt.l_plcy_key",
  "l_plcy_key_orig":"src.l_plcy_key",
  "dw_last_update_dt_orig":"src.dw_last_update_dt",
  "dw_create_dt":"tgt.dw_create_dt",
  "dw_row_start_dt":"tgt.dw_row_start_dt",
  "dw_row_end_dt":"tgt.dw_row_end_dt",
  "dw_last_update_dt": "current_timestamp()"
}

isSourceDeleteApplicable=True
sourceDeleteCondition="src.l_plcy_id IS NULL and tgt.l_plcy_id IS NOT NULL"
sourceDeleteExplicitColumnsMap={"dw_row_end_dt":"current_timestamp()","dw_row_active_ind":"'N'","dw_last_update_dt":"current_timestamp()","dw_batch_id":f"{batchKey}"}

#Converts Variables To Dict Format
scdInputs=dict((name, eval(name)) for name in ['seqPrefixValue','tgtTableSchema','tgtTableName','tgtFilterCondition','srcFilterCondition','joinColumnList','newInsertCondition','newInsertExplicitColumnMap','confChangeCondition','confChangeInsertExplicitColumnMap','confChangeUpdateExplicitColumnMap','axiomChangeDetectMd5ColumnList1','axiomChangeDetectMd5ColumnList2','axiomChangeCondition','axiomChangeUpdateExplicitColumnMap','axiomChangeInsertExplicitColumnMap','confHardStopCondition','confHardStopExplicitColumnMap','isSourceDeleteApplicable','sourceDeleteCondition',"sourceDeleteExplicitColumnsMap"])

In [0]:
stageName = 'Perform Type 2 Logic on SZ'
if previousStageCompletionFlag == True:
  try:
    loggerObject.info(f"########## {stageName} ##########")
    dfScd=spark.table(f"{INTERFACE_SCHEMA}.{INTERFACE_TABLE}")
    scdObject.performScdType2ConfAxiomDimAndFact(dfScd,scdInputs)
  except Exception as e:
    previousStageCompletionFlag = False
    errorMsg = f"FAILURE | Notebook run for {notebookName} - jobId: {notebookJobId} ,failed at : {stageName}"
    batchBalanceObject.closeBatch( batchSetCategory, batchSetSrcSysName, batchName, 'F')
    loggerObject.info(f"!!!!!!! Exception While {stageName} !!!!!!!")
    loggerObject.exception(e)
    print(e)
else:
    loggerObject.info(f"################# Skipping {stageName} #################")

In [0]:
srcTableSchema=CFM_SCHEMA
srcTableName="t_d_xll_policy"
srcFilterCondition=f"src.l_plcy_src_sys_cd = '{SRC_SYS_CD}' and src.dw_row_active_ind='N'"
tgtTableSchema=SZ_DELTA_SCHEMA
tgtTableName=TGT_SZ_TBL
tgtFilterCondition=f"tgt.l_plcy_src_sys_cd = '{SRC_SYS_CD}' and tgt.dw_row_active_ind='N' and tgt.l_plcy_key> 0"
joinColumnList=['l_plcy_key']
newInsertCondition="src.l_plcy_key IS NOT NULL and tgt.l_plcy_key IS NULL"
newInsertExplicitColumnMap={
  "l_plcy_authorization_status_ind":"src.l_plcy_authorisation_status_ind",
  "l_plcy_class_us_listed_ind":"src.l_us_listed"
}
#Converts Variables To Dict Format
InactiveLoadInputs=dict((name, eval(name)) for name in ['tgtTableSchema','tgtTableName','tgtTableName','tgtFilterCondition','srcFilterCondition','joinColumnList','newInsertCondition','newInsertExplicitColumnMap'])

In [0]:
stageName = 'Conformance N Load'
if previousStageCompletionFlag == True:
  try:
    loggerObject.info(f"########## {stageName} ##########")
    inputDf=spark.table(f"{srcTableSchema}.{srcTableName}")
    scdObject.performConfInactiveRecordLoad(inputDf,InactiveLoadInputs)
  except Exception as e:
    previousStageCompletionFlag = False
    errorMsg = f"FAILURE | Notebook run for {notebookName} - jobId: {notebookJobId} ,failed at : {stageName}"
    batchBalanceObject.closeBatch( batchSetCategory, batchSetSrcSysName, batchName, 'F')
    loggerObject.info(f"!!!!!!! Exception While {stageName} !!!!!!!")
    loggerObject.exception(e)
    print(e)
else:
    loggerObject.info(f"################# Skipping {stageName} #################")

In [0]:
stageName = 'Audit Balance and Close Batch'
if previousStageCompletionFlag == True:
  try:
    loggerObject.info(f"########## {stageName} ##########")  
    srcBalanceDf = spark.sql(srcBalanceSql)
    tgtBalanceDf = spark.sql(tgtBalanceSql) 
    balanceInd = batchBalanceObject.isBalanceMeasuresMatching(srcBalanceDf, tgtBalanceDf, batchSetCategory, batchSetSrcSysName, batchName)
    if balanceInd:
      batchBalanceObject.closeBatch(batchSetCategory, batchSetSrcSysName, batchName, "S")
    else:
      raise Exception(f"Balance Failure, Please Check Balance Table for further details")
  except Exception as e:
    previousStageCompletionFlag = False
    errorMsg = f"FAILURE | Notebook run for {notebookName} - jobId: {notebookJobId} ,failed at : {stageName}"
    batchBalanceObject.closeBatch( batchSetCategory, batchSetSrcSysName, batchName, 'F')
    loggerObject.info(f"!!!!!!! Exception While {stageName} !!!!!!!")
    loggerObject.exception(e)
    print(e)
else:
    loggerObject.info(f"################# Skipping {stageName} #################")

In [0]:
%run /DATA_ENGINEERING/2_AXIOM/1_SHARED/UTILITIES/post_job_execution_utility

In [0]:
if previousStageCompletionFlag == True:
  dbutils.notebook.exit("SUCCESS")
else:
  dbutils.notebook.exit(errorMsg)